In [7]:
%pip install redis

  Using cached redis-4.3.5-py3-none-any.whl (248 kB)
  Using cached async_timeout-4.0.2-py3-none-any.whl (5.8 kB)
Note: you may need to restart the kernel to use updated packages.


In [50]:

import main
import redis
import json
import os
from dotenv import dotenv_values

config = dotenv_values(".env")  # config = {"USER": "foo", "EMAIL": "foo@example.org"}


In [51]:
config['CLIENT_ID']

'VG1iak5nSS1HeXAwUDk5TjhCX2c6MTpjaQ'

In [52]:

# You can now set a variable called twitter, which calls the make_token function to create a new access token. You will also need to obtain the environment variables for client ID and client secret. 
twitter = main.make_token()
client_id = config['CLIENT_ID']
client_secret = config['CLIENT_SECRET']
token_url = "https://api.twitter.com/2/oauth2/token"


In [53]:

# Now, you can obtain the access token from Redis, which is saved corresponding with the value of token. You will also need to decode the token and replace the quotes. You can save it into a JSON object and work with it later.
t = main.r.get("token")
print(t)

b"{'token_type': 'bearer', 'expires_in': 7200, 'access_token': 'OHNULUhTU0s5X1B6SmhNVHhpOFFMTXpZdjlFWklBRzRJMmRKTnoyUHRTbmpfOjE2NjkzMjQ0NzE1NDM6MTowOmF0OjE', 'scope': ['tweet.write', 'users.read', 'tweet.read', 'offline.access'], 'refresh_token': 'eHdRSVo1Z3phcFR4bDZtcTVxU0llTHRHSW1yRDFBUEV2Mm53TXF6VEJLSDZnOjE2NjkzMjQ0NzE1NDM6MToxOnJ0OjE', 'expires_at': 1669331671.536252}"


In [54]:
bb_t = t.decode("utf8").replace("'", '"')
print(bb_t)

{"token_type": "bearer", "expires_in": 7200, "access_token": "OHNULUhTU0s5X1B6SmhNVHhpOFFMTXpZdjlFWklBRzRJMmRKTnoyUHRTbmpfOjE2NjkzMjQ0NzE1NDM6MTowOmF0OjE", "scope": ["tweet.write", "users.read", "tweet.read", "offline.access"], "refresh_token": "eHdRSVo1Z3phcFR4bDZtcTVxU0llTHRHSW1yRDFBUEV2Mm53TXF6VEJLSDZnOjE2NjkzMjQ0NzE1NDM6MToxOnJ0OjE", "expires_at": 1669331671.536252}


In [55]:
data = json.loads(bb_t)
print(data)

{'token_type': 'bearer', 'expires_in': 7200, 'access_token': 'OHNULUhTU0s5X1B6SmhNVHhpOFFMTXpZdjlFWklBRzRJMmRKTnoyUHRTbmpfOjE2NjkzMjQ0NzE1NDM6MTowOmF0OjE', 'scope': ['tweet.write', 'users.read', 'tweet.read', 'offline.access'], 'refresh_token': 'eHdRSVo1Z3phcFR4bDZtcTVxU0llTHRHSW1yRDFBUEV2Mm53TXF6VEJLSDZnOjE2NjkzMjQ0NzE1NDM6MToxOnJ0OjE', 'expires_at': 1669331671.536252}


In [56]:

# Since access tokens in OAuth 2.0 only stay valid for two hours, you will need to refresh your token. Refresh tokens typically stay valid for about six months.
refreshed_token = twitter.refresh_token(
    client_id=client_id,
    client_secret=client_secret,
    token_url=token_url,
    refresh_token=data["refresh_token"],
)


UnauthorizedClientError: (unauthorized_client) Missing valid authorization header

In [ ]:

# To save the token, you will need to ensure it has the proper quotations around it and load into a JSON object before you can save it back into Redis with the value of token.

st_refreshed_token = '"{}"'.format(refreshed_token)
j_refreshed_token = json.loads(st_refreshed_token)
main.r.set("token", j_refreshed_token)


In [ ]:

# After saving the newly refreshed token back into Redis, now you can obtain a new cat fact from the cat fact API, pass that into a JSON payload, and Tweet.

caty_fact = main.parse_cat_fact()
payload = {"text": "{}".format(caty_fact)}
main.post_tweet(payload, refreshed_token)